In [23]:
import requests
from bs4 import BeautifulSoup
import time 
from tqdm.notebook import tqdm  
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
import os 

In [26]:
os.listdir('../monin')

['monin_drinks_urls.json', 'temp']

In [ ]:
browser = webdriver.Firefox()
browser.get('https://www.monin.com/us/recipes/drinks')

In [ ]:
def find_drinks_on_page(browser, page_url, page_number): 
    drink_urls = [] 

    browser.get(f'{page_url}&page={page_number}')
    browser.implicitly_wait(5) 
    time.sleep(1)   

    browser.execute_script("window.scrollBy(0, document.body.scrollHeight);")       
    browser.implicitly_wait(3) 
    time.sleep(1)

    browser.execute_script("window.scrollBy(0, document.body.scrollHeight);") 
    browser.implicitly_wait(2)
    time.sleep(4)
    
    for element in browser.find_elements(By.TAG_NAME, 'ul'): 
        if element.get_attribute('class') != 'ProductListPage CategoryProductList-Page CategoryProductList-Page_layout_grid': 
            continue

        for drink in element.find_elements(By.TAG_NAME, 'a'): 
            drink_urls.append(drink.get_attribute('href') )

    return drink_urls

def parse_pages(browser, pages_base_url, pages_number): 
    urls = [] 

    for page_number in tqdm(range(1, pages_number + 1)):
        page_urls = find_drinks_on_page(browser, pages_base_url, page_number)

        if len(page_urls) == 0: 
            print('Error occured in:', f'\n\t{pages_base_url}', f'\n\ton page: {page_number}')

        urls += page_urls

    return urls


In [ ]:
pages = {
    # parsed
    'lemonade': [
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:18', 9)
    ], 

    # parsed
    'coffee':  [
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:14', 16)
    ],

    # 'milkshake': [
    #     ('', 1)
    # ],

    # 'hot_tea': [
    #     ('', 1)
    # ],

    # parsed
    'cold_tea': [
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:33', 5)
    ],

    'alcohol': [
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:16', 32)
    ],

    'smoothie': [
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:558', 3), 
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:724', 2), 
    ],      

    'punch': [
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:163', 2)
    ],    
}

In [ ]:
drink_urls = {} 

for category in tqdm(pages): 
        for category_url, pages_number in pages[category]:      
                drink_urls[category] = drink_urls.get(category, []) + parse_pages(browser, category_url, pages_number)

In [ ]:
for category in drink_urls: 
    print(category, len(drink_urls[category]))

In [ ]:
with open('../monin/monin_drinks_urls.json', 'w') as file:
    json.dump(drink_urls, file)

In [ ]:
# with open('../monin/monin_drinks_urls.json') as file:
#     drink_urls = json.load(file)

In [30]:
for category in drink_urls: 
    print(category, len(drink_urls[category]))

lemonade 195
coffee 384
cold_tea 108
alcohol 765
smoothie 93
punch 27


In [36]:
cache = {}

In [37]:
drinks = [] 

for category in tqdm(drink_urls): 
    urls = drink_urls[category]

    if 'category' not in os.listdir('../monin'): 
        os.mkdir(f'../monin/{category}')

    for url in tqdm(urls): 
        browser.get(url)
        browser.implicitly_wait(4)
        time.sleep(3.5)    

        for script in browser.find_elements(By.TAG_NAME, 'script'):
            if script.get_attribute('id') == 'recipe-data-snippet':     
                drink_data = json.loads(script.get_attribute('innerHTML'))
                drink_name = drink_data['name']

                image_link = drink_data['image']
                image_ext = image_link.split('.')[-1]
                timestamp = str(time.time()).replace('.', '_')

                recipie =  '\n'.join(list(map(lambda x: x.replace('</sup>', '').replace('<sub>', ''), drink_data['recipeIngredient']))) + '\n' * 2 + '\n'.join(drink_data['recipeInstructions'])

                image_request = requests.get(image_link)
                
                with open(f'../monin/{category}/{timestamp}.{image_ext}', mode='wb') as file: 
                    file.write(image_request.content)

                drinks.append({
                    'name': drink_name, 
                    'recipie': recipie,
                    'category': category, 
                    'image_name': f'{timestamp}.{image_ext}', 
                    'image_ext': image_ext, 
                    'image_url': image_link
                })

                # print({
                #     'name': drink_name, 
                #     'recipie': recipie,
                #     'category': category, 
                #     'image_name': f'{timestamp}.{image_ext}', 
                #     'image_ext': image_ext, 
                #     'image_url': image_link
                # })

                # print('-'*20 + '\n\n')
                break
    cache[category] = drinks.copy()

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/384 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/765 [00:00<?, ?it/s]

TimeoutException: Message: Navigation timed out after 300000 ms
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:740:5
bail@chrome://remote/content/marionette/sync.sys.mjs:211:19


In [45]:
import csv


with open('../drink_recipies_monin.csv', 'w', newline='') as csvfile:
    fieldnames = ['name', 'recipie', 'category', 'image_name', 'image_ext', 'image_url']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(drinks)

In [47]:
import pandas as pd

pd.read_csv('../drink_recipies_monin.csv')['category'].value_counts()

category
coffee      384
lemonade    195
cold_tea    108
Name: count, dtype: int64